Part 0

DONE: Watch Tutorial, adapt crossentropy loss

Part 1a

DONE: Create dense network classifying cancerous/not cancerous

Part 1b

DONE:
-Understand U-Net Architecture
-Understand concatenation of inputs
-Understand ConvTranspose
-The simplest network wouldn't work well because it would limit the number of features that the convolution is able to recognize. By reducing down the image height and width, and increasing the number of channels, the network is able to recognizing many features of varying size.  

Part 2 & Part 3

Done: Plot training & validation loss, and training and validation accuracy, generate prediction on pos_test_000072.png

![alt text](https://thleats-bucket.s3.us-east-2.amazonaws.com/CS/256_v4.gif)

In [0]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms, utils, datasets
from tqdm import tqdm
import scipy
from scipy import signal
from torch.nn.parameter import Parameter
import pdb

import os
import gzip
import tarfile
import gc

####################Paramters#####################
###################################################

epochs=20

batch_size=15

save_model=0

save_images=1

aws=0 #don't change this


###################################################
###################################################

def count_parameters(model):
    total_param = 0
    for name, param in model.named_parameters():
        if param.requires_grad:
            num_param = np.prod(param.size())
            total_param += num_param
    return total_param
  

assert torch.cuda.is_available() # You need to request a GPU from Runtime > Change Runtime Type
  

########################Convolution Network Class###############################
parameters=[]
class ConvNetwork(nn.Module):
  def __init__(self):
    super(ConvNetwork,self).__init__()
    c=3
    out =2
    channels=64
    #####Create the first block
    self.conv1=nn.Conv2d(c,channels,(3,3),padding=(1,1))
    self.conv2=nn.Conv2d(channels,channels,(3,3),padding=(1,1))
    
    
    #####Copy to the last block   
    
    #####Create the second block
    self.conv4=nn.Conv2d(channels,channels*2,(3,3),padding=(1,1))
    self.conv5=nn.Conv2d(channels*2,channels*2,(3,3),padding=(1,1))
    
    
    #####Copy to the 2nd to last block
    
    #####Create the third block
    self.conv7=nn.Conv2d(channels*2,channels*4,(3,3),padding=(1,1))
    self.conv8=nn.Conv2d(channels*4,channels*4,(3,3),padding=(1,1))
    
    
    #####Copy to the 3rd to last block
    
    #####Create the fourth block
    self.conv10=nn.Conv2d(channels*4,channels*8,(3,3),padding=(1,1))
    self.conv11=nn.Conv2d(channels*8,channels*8,(3,3),padding=(1,1))
    
    
    #####Copy to the 4th to last block
    
    #####Create the fifth block
    self.conv13=nn.Conv2d(channels*8,channels*16,(3,3),padding=(1,1))
    self.conv14=nn.Conv2d(channels*16,channels*16,(3,3),padding=(1,1))
   
    
    #####Upsample to sixth block
    self.conv15=nn.ConvTranspose2d(channels*16,channels*8,2,stride=2,padding=0)
    self.conv16=nn.Conv2d(channels*16,channels*8,(3,3),padding=(1,1))
    self.conv17=nn.Conv2d(channels*8,channels*8,(3,3),padding=(1,1))

    
    #####Upsabmle to seventh block
    self.conv18=nn.ConvTranspose2d(channels*8,channels*4,(2,2),padding=0,stride=2)
    self.conv19=nn.Conv2d(channels*8,channels*4,(3,3),padding=(1,1))
    self.conv20=nn.Conv2d(channels*4,channels*4,(3,3),padding=(1,1))
   
    
    #####Upsample to eigth block
    self.conv21=nn.ConvTranspose2d(channels*4,channels*2,(2,2),padding=0,stride=2)
    self.conv22=nn.Conv2d(channels*4,channels*2,(3,3),padding=(1,1))
    self.conv23=nn.Conv2d(channels*2,channels*2,(3,3),padding=(1,1))

    
    #####Upsample to ningth block
    self.conv24=nn.ConvTranspose2d(channels*2,channels,(2,2),padding=0,stride=2)
    self.conv25=nn.Conv2d(channels*2,channels,(3,3),padding=(1,1))
    self.conv26=nn.Conv2d(channels,channels,(3,3),padding=(1,1))
    self.conv27=nn.Conv2d(channels,out,(1,1),padding=(0,0))
    
  
    self.activation = nn.ReLU()
  
  def forward(self,x):
    n,c,h,w = x.size()
    memory=[]
    
    o1=self.activation(self.conv1(x))
    #print(o1.size())
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    o2=self.activation(self.conv2(o1))#o2 to o24
    #print(o2.size())
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    size_tensor=o2.size()
    variable=size_tensor[2]/2
    
    ##
    #torch.cat((o1,o24),1)
    #pdb.set_trace()
    m=nn.AdaptiveMaxPool2d(int(variable))
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    o3=m(o2)
    #print(o3.size())
    o4=self.activation(self.conv4(o3))
    #print(o4.size())
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    o5=self.activation(self.conv5(o4))#o5 to o21
    #print(o5.size())
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    
    size_tensor=o5.size()
    variable=size_tensor[2]/2
    ##
    m=nn.AdaptiveMaxPool2d(int(variable))
    
    o6=m(o5)
    #print(o6.size())
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    o7=self.activation(self.conv7(o6))
    #print(o7.size())
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    o8=self.activation(self.conv8(o7))#o8 to o18
    #print(o8.size())
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    
    size_tensor=o8.size()
    variable=size_tensor[2]/2
    ##
    m=nn.AdaptiveMaxPool2d(int(variable))
    o9=m(o8)
    #print(o9.size())
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    o10=self.activation(self.conv10(o9))
    #print(o10.size())
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    o11=self.activation(self.conv11(o10))#o11 to o15
    #print(o11.size())
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    
    size_tensor=o11.size()
    variable=size_tensor[2]/2
    ##
    m=nn.AdaptiveMaxPool2d(int(variable))
    o12=m(o11)
    #print(o12.size())
    memory.append(torch.cuda.memory_allocated(0)/1e9)    
    o13=self.activation(self.conv13(o12))
    #print(o13.size())
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    o14=self.activation(self.conv14(o13))
    #print(o14.size())
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    ##
    
    #m=nn.Upsample(28)
    
    o15=self.conv15(o14)
    #print(o15.size())
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    o15=torch.cat((o11,o15),1)
    #print(o15.size())
    #
    o16=self.activation(self.conv16(o15))
    #print(o16.size())
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    o17=self.activation(self.conv17(o16))
    #print(o17.size())
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    ##
    #m=nn.Upsample(28)
    o18=self.conv18(o17)
    #print(o18.size())
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    o18=torch.cat((o8,o18),1)
    #print(o18.size())
    o19=self.activation(self.conv19(o18))
    #print(o19.size())
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    o20=self.activation(self.conv20(o19))
    #print(o20.size())
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    ##
    #m=nn.Upsample(28)
    #pdb.set_trace()
    o21=self.conv21(o20)
    #print(o21.size())
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    o21=torch.cat((o5,o21),1)
    #print(o21.size())
    o22=self.activation(self.conv22(o21))
    #print(o22.size())
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    o23=self.activation(self.conv23(o22))
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    #print(o23.size())
    ##
    
    #m=nn.Upsample(28)
    o24=self.conv24(o23)
    #print(o24.size())
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    o24=torch.cat((o2,o24),1)
    #print(o24.size())
    o25=self.activation(self.conv25(o24))
    #print(o25.size())
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    o26=self.activation(self.conv26(o25))
    #print(o26.size())
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    
    o27=self.conv27(o26)
    #print(o27.size())
    memory.append(torch.cuda.memory_allocated(0)/1e9)
    #pdb.set_trace()
    #fig, ax = plt.subplots()
    #plt.plot(memory)
    #ax=plt.gca()
    #loss_v=ax.plot(memory)
    #loss_t=ax.plot(iterations,losses,label='Training Loss')
    #acc=ax.plot(iterations,acc_vec,label='Accuracy Validation')
    #acc_t=ax.plot(iterations,acc_vec_t,label='Accuracy Training')



    #ax.legend()


    #ax.set(xlabel='U-net layer #', ylabel='GPU Memory Usage - GB',
           #title='U-net Layers vs GPU Memory usage')
    #ax.grid()
    
    #plt.show()
    return o27.squeeze(2).squeeze(2)
    

        
    
class CancerDataset(Dataset):
  def __init__(self, root, download=True, size=256, train=True):
    if download and not os.path.exists(os.path.join(root, 'cancer_data')):
      datasets.utils.download_url('https://nolans-cs-bucket.s3-us-west-1.amazonaws.com/cancer_data.tar.gz', root, 'cancer_data.tar.gz', None)
      self.extract_gzip(os.path.join(root, 'cancer_data.tar.gz'))
      self.extract_tar(os.path.join(root, 'cancer_data.tar'))
    
    postfix = 'train' if train else 'test'
    root = os.path.join(root, 'cancer_data', 'cancer_data')
    self.dataset_folder = torchvision.datasets.ImageFolder(os.path.join(root, 'inputs_' + postfix) ,transform = transforms.Compose([transforms.Resize(size),transforms.ToTensor()]))
    self.label_folder = torchvision.datasets.ImageFolder(os.path.join(root, 'outputs_' + postfix) ,transform = transforms.Compose([transforms.Resize(size),transforms.ToTensor()]))

  @staticmethod
  def extract_gzip(gzip_path, remove_finished=False):
    print('Extracting {}'.format(gzip_path))
    with open(gzip_path.replace('.gz', ''), 'wb') as out_f, gzip.GzipFile(gzip_path) as zip_f:
      out_f.write(zip_f.read())
    if remove_finished:
      os.unlink(gzip_path)
  
  @staticmethod
  def extract_tar(tar_path):
    print('Untarring {}'.format(tar_path))
    z = tarfile.TarFile(tar_path)
    z.extractall(tar_path.replace('.tar', ''))

  def __getitem__(self,index):
    img = self.dataset_folder[index]
    label = self.label_folder[index]
    return img[0],label[0][0]
  
  def __len__(self):
    return len(self.dataset_folder)
    #return 30
    
def compute_accuracy(y_hat,y_truth):
  test=torch.argmax(y_hat,dim=1)
  y_truth=y_truth.long()
  
  test2=test==y_truth
  chn=test.size()
  pix=test.size(2)
  print("pix is:"+ str(pix))
  out=torch.sum(test2).item()/(pix*pix*chn[0])
  
  return out


############### #LOADING AND TRAINING  ################################
########################################################################
########################################################################
########################################################################
    
    

##load data
train_dataset=CancerDataset('/tmp/cancer_train',train=True)
val_dataset=CancerDataset('/tmp/cancer_test',train=False)

#create model
model = ConvNetwork()
#send model to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if torch.cuda.device_count() > 1:
  print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
  model = nn.DataParallel(model)
  aws=1


model.to(device)
#model.cuda()

#define the objective function
objective = torch.nn.CrossEntropyLoss()

#define the optimizer
optimizer = optim.Adam(model.parameters(),lr=1e-4)



losses=[]
losses_valid=[]
loss_mean=[]
acc_vec=[]
acc_epoch=[]
accuracy_v=[]
accuracy_t=[]
acc_vec_t=[]
acc_vec_loop=[]
acc_v_loop=[]
memory_t=[]
memory_v=[]
memory_t_temp=[]
memory_v_temp=[]



if aws==1:
  workers=64
else:
  workers=2
  
  
 #define the batch loader for training 
train_loader = DataLoader(train_dataset,
                           batch_size=batch_size,
                           pin_memory=True,num_workers=workers)  
  
  #define teh batch loader for validation
valid_load = DataLoader(val_dataset,
                        batch_size=round(len(val_dataset)/len(train_loader)),
                        pin_memory=True,num_workers=workers)

#break
gc.collect()
torch.cuda.empty_cache()
############################ START OF EPOCH LOOP ###############################

epoch_counter=1
for epoch in range(epochs):
  loop = tqdm(total=len(train_loader), position = 0)
  loop2 = tqdm(total=len(valid_load), position = 0)
  
  #########################START OF BATCH LOOP #############################
  
  ####################TRAINING
  for batch in train_loader:
    x=batch[0]
    y_truth=batch[1]
    x=x.cuda()
    y_truth=y_truth.cuda()
    optimizer.zero_grad()
    y_hat=model(x)
    
    acc = np.mean(compute_accuracy(y_hat, y_truth))
    dims = y_truth.size()
    y_truth = y_truth.long()
    loss = objective(y_hat, y_truth)
    loss.backward()
    optimizer.step()
    losses.append(loss)
    acc_vec_loop.append(acc)
    memory_t_temp=torch.cuda.memory_allocated(0)/1e9
    loop.update(1)
    
    
  memory_t.append(memory_t_temp)
  gc.collect()
  acc_vec.append(np.mean(acc_vec_loop))
  loop.set_description('training loss:{:.4f}'.format(loss.item()))

  #####################VALIDATION
  for x_valid,y_truth_valid in valid_load:
    
    x_valid,y_truth_valid = x_valid.cuda(async=True),y_truth_valid.cuda(async=True)
    
    with torch.no_grad():
      y_nohat=model(x_valid)
      
      acc_v = np.mean(compute_accuracy(y_nohat,y_truth_valid))
      y_truth_valid=y_truth_valid.long()
      temp=objective(y_nohat,y_truth_valid)
      
    #accuracy_v.append(acc_v)
    temp_cpu=temp.cpu()
    #loss_mean.append(np.mean(temp))
    
    losses_valid.append(temp_cpu.item())
    acc_v_loop.append(np.mean(acc_v))
    memory_v_temp=(torch.cuda.memory_allocated(0)/1e9)
    gc.collect()
    loop2.update(1)
  memory_v.append(memory_v_temp)
  accuracy_v.append(np.mean(acc_v_loop))
  loop2.set_description('validation loss:{:.4f}'.format(losses_valid[-1]))
  
  
  ##################### PART 3 OF LAB - IMAGE 72
  if save_images==1:
  
    for things in valid_load:
      break
    original=things[0][0]
    original=original.unsqueeze(0).cuda()

    truth=things[1][0]
    truth_cuda=truth.cuda()

    yhat_eval=model(original)
    yhat_eval_sm=torch.nn.functional.softmax(yhat_eval, dim=1)
    yhat_mask=torch.argmax(yhat_eval_sm,dim=1)
    yhat_mask=yhat_mask.squeeze(0).cuda()

    accuracy_image = compute_accuracy(yhat_eval,truth_cuda)
    original.squeeze_(0)
    original_cpu=original.cpu()
    r=original_cpu[0]
    g=original_cpu[1]
    b=original_cpu[2]

    image=np.zeros((1024,1024,3))
    image[:,:,0]=r
    image[:,:,1]=g
    image[:,:,2]=b
    
    fig=plt.figure(figsize=(15,10))
    fig.suptitle('U-Net Data Plots - 1024x1024 pixels',fontsize=25)
    ax1=fig.add_subplot(231)
    
    plt.imshow(yhat_mask.cpu())
    location=yhat_mask.size(1)/2
    acc_string=str(accuracy_image)
    plt.text(location,location*1.5,'accuracy = ' + acc_string[0:5],verticalalignment='center',horizontalalignment='center',color='w')
    plt.text(location,location*1.75,'epoch ' + str(epoch_counter),verticalalignment='center',horizontalalignment='center',color='w')
    

    ax2=fig.add_subplot(232)
    
    plt.subplot(232)
    plt.text(location,location*1.5,'truth',verticalalignment='center',horizontalalignment='center',color='w')
    truth_cpu=truth.cpu()
    plt.imshow(truth_cpu)
    
   
    ax3=fig.add_subplot(233)
    
    plt.text(location,location*1.5,'original',verticalalignment='center',horizontalalignment='center')
    plt.imshow(image)
    
    
    ax4=fig.add_subplot(234)
    
    iterations=range(0,len(losses),1) 
    valid_iterations=range(0,len(losses_valid),1)
    plt.plot(valid_iterations,losses_valid,label='Validation Loss')
    plt.plot(iterations,losses,label='Training Loss')
    ax4.legend(loc="upper right")
    ax4.title.set_text('Loss vs Iterations')
    ax4.set_xlabel('iterations')
    ax4.set_ylabel('loss')
    
    
    ax5=fig.add_subplot(235)
    
    iterations_t=range(0,len(acc_vec),1)
    iterations_v=range(0,len(accuracy_v),1)
    plt.plot(iterations_t,acc_vec,label='Validation Accuracy')
    plt.plot(iterations_v,accuracy_v,label='Training Accuracy')
    ax5.legend(loc="lower right")
    ax5.title.set_text('Accuracy vs Iterations')
    ax5.set_xlabel('iterations')
    ax5.set_ylabel('accuracy')
    
    
    #ax6=fig.add_subplot(236)
    #epochs_memory=range(0,len(memory_t),1)
    #plt.plot(epochs_memory,memory_t,label='training memory')
    #plt.plot(epochs_memory,memory_v,label='validation memory')
    #ax6.legend(loc="lower right")
    #ax6.set_xlabel('epochs')
    #ax6.title.set_text('GPU memory usage vs epochs')
    #ax6.set_ylabel('GPU memory usage-GB')
    
    plt.subplots_adjust(wspace=0.25)
    if aws==0:
      from google.colab import drive
      drive.mount('/content/drive')
      with open('/content/drive/My Drive/image_' + str(epoch_counter) + '.png', 'w') as f:
        out=plt.savefig('/content/drive/My Drive/image_' + str(epoch_counter) + '.png')
    if aws==1:
      with open('/home/ubuntu/image_' + str(epoch_counter) + '.png', 'w') as f:
        out=plt.savefig('/home/ubuntu/image_' + str(epoch_counter) + '.png')

    
    print("images saved")
  
  
  epoch_counter=epoch_counter+1   
  loop.close()
 #########################END OF EPOCH LOOP ############################# 






######################## POSTPROCESSING ################################
########################################################################
########################################################################
########################################################################

#################SAVE MODEL
if save_model==1:
  with open('/home/ubuntu/foo.txt', 'w') as f:
    out=torch.save(model.state_dict(),'/home/ubuntu/foo.txt')






0it [00:00, ?it/s]

100%|█████████▉| 2749071360/2751290623 [00:39<00:00, 74118346.72it/s]

Extracting /tmp/cancer_train/cancer_data.tar.gz


2751291392it [00:50, 74118346.72it/s]                                

Untarring /tmp/cancer_train/cancer_data.tar


...

 57%|█████▋    | 31/54 [01:43<01:16,  3.31s/it]

pix is:256


 59%|█████▉    | 32/54 [01:46<01:12,  3.31s/it]

pix is:256


 61%|██████    | 33/54 [01:49<01:09,  3.30s/it]